<a href="https://colab.research.google.com/github/Nayoung-young/lec-ecommerce-clone/blob/main/ch03_logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#_01. 분석 목적 

- logsitic regression: 이진분류를 예측 
- y : 광고 클릭 여부, x : 고객에 대한 정보 

#_02. 모듈, 데이터 로딩 및 데이터 확인 

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns

In [ ]:
data = pd.read_csv('ecommerce.csv')

In [ ]:
data.head(10)

#_03. 결측치 확인 및 처리 

#_04. Train, Test set 나누기 

#_05. Logistic Regression Modeling 

#_06. Evaluation 